<a href="https://colab.research.google.com/github/Esther021004/Ai_service_design/blob/main/%EC%97%90%EB%B8%8C%EB%A6%AC%ED%83%80%EC%9E%84_%EA%B0%95%EC%9D%98%ED%8F%89_%EC%88%98%EC%A7%91_%ED%81%AC%EB%A1%A4%EB%9F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**에브리타임 강의평 수집 크롤러**

에브리타임에서 모든 강의평을 수집할 수 있는 파이썬 코드이다.

##**1. 수집 과정 설명**

(1) 학교 개설강좌의 교과목명과 교수명을 수집한 파일을 만든다.    
(2) 에브리타임 로그인을 Selenium WebDriver로 자동화한다.     
(3) (1)의 파일에서 [교과목명과 교수] 리스트를 로딩한다.      
(4) 각 강의에 대해 반복 작업을 수행       
&emsp;a. 강의실 메인 페이지 이동      
&emsp;b. 검색창에 강의명 입력 -> 검색 실행      
&emsp;c. 검색 결과 중에서 교수명까지 일치하는 항목 찾기      
&emsp;d. 일치하는 강의 클릭 -> 강의평 탭 클릭      
&emsp;e. 강의평 (max 20개) 수집      
&emsp;f. 수집한 텍스트를 리스트에 저장      
&emsp;g. 다음 강의를 위해 메인 페이지로 복귀

##**2. 필수 패키지???**

In [ ]:

import pandas as pd
import time


##**3. 코드 설명**

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 1. 드라이버 설정
options = Options()
options.add_argument("user-agent=Mozilla/5.0")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(10)

# 2. 로그인
driver.get("https://everytime.kr/login")
time.sleep(2)
driver.find_element(By.NAME, "id").send_keys("아이디 입력하세요") #자신의 에타 아이디 입력
driver.find_element(By.NAME, "password").send_keys("비밀번호 입력하세요") #자신의 에타 비밀번호 입력
driver.find_element(By.NAME, "password").send_keys(Keys.RETURN)
time.sleep(3)
print("✅ 로그인 완료")

# 3. 강의실 메인 이동
driver.get("https://everytime.kr/lecture")
time.sleep(3)

# 4. CSV 불러오기
lecture_df = pd.read_csv("강의명_교수명_리스트.csv")  #자신의 학교 개설강좌의 교과목명, 교수 데이터 파일을 넣기
lecture_df.columns = lecture_df.columns.str.strip()
lecture_df = lecture_df.dropna(subset=["교과목명", "교수"])  # 위의 파일에 열 부분 이름이 "교과목명", "교수"이어야함 / 파일대로 수정해도 됨
lecture_list = lecture_df[["교과목명", "교수"]].values.tolist()

# 5. 전체 강의 검색 및 강의평 수집
data = []

for subject, professor in lecture_list:
    try:
        # ✅ 강의실 메인으로 복귀
        driver.get("https://everytime.kr/lecture")
        time.sleep(2)

        print(f"🔍 검색 중: {subject} / {professor}")

        # 검색창 찾기
        search_box = driver.find_element(By.CSS_SELECTOR, 'input[type="search"]')
        search_box.clear()
        search_box.send_keys(subject)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)

        # 검색 결과에서 일치 항목 찾기
        results = driver.find_elements(By.CSS_SELECTOR, "a.lecture")
        matched = False
        for r in results:
            try:
                title = r.find_element(By.CSS_SELECTOR, ".highlight").text.strip()
                professor_name = r.find_element(By.CSS_SELECTOR, ".professor").text.strip()
            except:
                continue  # 교수명이 없는 블록은 건너뜀

            if subject.replace(" ", "") in title.replace(" ", "") and professor.replace(" ", "") in professor_name.replace(" ", ""):
                print(f"✅ 매칭: {title} / {professor_name}")
                lecture_url = r.get_attribute("href")
                driver.get(lecture_url)
                time.sleep(2)
                matched = True
                break

        if not matched:
            print(f"❌ 강의 미매칭: {subject} / {professor}")
            continue

        # 강의평 탭 클릭
        try:
            review_tab = driver.find_element(By.LINK_TEXT, "강의평")
            review_tab.click()
            time.sleep(2)
        except:
            print("❌ 강의평 탭 클릭 실패")
            driver.back()
            time.sleep(1)
            continue

        # 강의평 수집 (최대 20개)
        reviews = driver.find_elements(By.CSS_SELECTOR, ".text")
        full_text = "\n".join([r.text for r in reviews])
        print(f"📝 {len(reviews)}개 강의평 수집됨")

        data.append([subject, professor, full_text.strip()])
        driver.back()
        time.sleep(2)

    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")
        continue

# 6. 수집 결과 저장
df = pd.DataFrame(data, columns=["과목명", "교수명", "강의평"])
df.to_csv("성신여대_전체강의_강의평.csv", index=False, encoding="utf-8-sig")
print("✅ 전체 강의 크롤링 완료 및 CSV 저장!")

# 7. 드라이버 종료
driver.quit()
